### Given a currency basket, start date and end date, we estimate the weights and analyse the performance.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import coint_trading as ct
import utils as ut
import plotting as myplt
import data_io as myio
import mean_reversion as mrev

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual
from datetime import datetime

from ta.momentum import RSIIndicator

import warnings
warnings.filterwarnings("ignore")

### User Specification

In [ ]:
FROM = '2010-08-23'
name_list = ['AUDUSD Curncy', 'CADUSD Curncy']

FROM = '2017-07-23'
name_list = ['GBPUSD Curncy', 'AUDUSD Curncy', 'NZDUSD Curncy', 'CADUSD Curncy']

FROM = '2016-10-23'
name_list = ['EURUSD Curncy', 'GBPUSD Curncy', 'AUDUSD Curncy', 'NZDUSD Curncy', 'CNHUSD Curncy']

TODAY = '2024-01-24'

### Read in data and perform estimation using the latest data

In [ ]:
df_fx_spot, df_fx_ivol = myio.read_all_fx_data('../data/bloomberg fx data sheet.xlsx')
df_fx_spot = df_fx_spot.loc[FROM:TODAY]
df_fx_name_list_XXXUSD = df_fx_spot[name_list]

res_estimation = ct.johansen_test_estimation(df_fx_name_list_XXXUSD, name_list, 0, 1)
res_diag = ct.johansen_test_diag(res_estimation, df_fx_name_list_XXXUSD, name_list, True, 0, 1)

weights_XXXUSD = res_estimation['weights']
basket = res_diag['Johansen Basket']
#------------------------------------------------------------------------------------
LOWER_THRESHOLD = 2.2
HIGHER_THRESHOLD = 2.5

mean_rev_ts = mrev.MeanRevTimeSeries(basket)
long_run_mean = mean_rev_ts.get_mean_rev_level()
historical_std = mean_rev_ts.get_stdev()

df_MeanRevStats = mrev.compute_sharpe_and_buysell_signal_multi_period(basket,
                                                                      long_run_mean,
                                                                      historical_std,
                                                                      mean_rev_ts.get_mean_rev_rate_in_days(),                                                                      
                                                                      mean_rev_ts.get_half_life_in_days(),
                                                                      LOWER_THRESHOLD,
                                                                      HIGHER_THRESHOLD)

basket_x_days_hist_rtns = ut.compute_x_day_historical_returns(basket)
#------------------------------------------------------------------------------------
df_buy_sell_signal = df_MeanRevStats[['Buy Signal', 'Sell Signal']]
df_Sharpe = df_MeanRevStats[['Sharpe Ratio']]

df_hist_vol = pd.DataFrame(ut.compute_daily_hist_normal_vol(basket, 15))
df_rsi = pd.DataFrame(RSIIndicator(basket, 14).rsi())

shock_in_market_conv_pips = 10
dBasket_SD = ut.basket_SD_change_due_to_fx_spot_change(df_fx_name_list_XXXUSD, 
                                                       weights_XXXUSD, 
                                                       shock_in_market_conv_pips,
                                                       long_run_mean,
                                                       historical_std
                                                       )
#print(dBasket_SD)

ts = (basket - long_run_mean)/historical_std
df_diff = pd.DataFrame(ts.diff())
df_diff = df_diff.rename(columns={"Basket": "One day diff in SD"})
print("--------------------------------------------------")
print(df_diff.quantile([0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]))

### Time Seires of Basket with z-score using historical std (for trading signal)

In [ ]:
@interact_manual
def show_articles_more_than(start_date = widgets.DatePicker(value=pd.to_datetime('2023-02-01')),
                            end_date   = widgets.DatePicker(value=pd.to_datetime(TODAY)),
                            vert_line_date = widgets.DatePicker(value=pd.to_datetime('2021-10-10')),
                            z_lower = (-8, 2, 0.5), z_upper = (-2, 8, 0.5), quantile = (1,10)):        
    xlim_range = (start_date, end_date)   
    ylim_range = (long_run_mean + z_lower * historical_std, long_run_mean + z_upper * historical_std)
    myplt.plot_trading_signal_graphs(basket, long_run_mean, historical_std, 
                                     df_buy_sell_signal, xlim_range, ylim_range, vert_line_date)      
    myplt.normal_plot(df_hist_vol, xlim_range, vert_line_date, quantile, 'b-o') 
    myplt.normal_plot(df_Sharpe, xlim_range, vert_line_date, quantile, 'g-o')  
    myplt.normal_plot(df_rsi, xlim_range, vert_line_date, quantile, 'r-o')
    myplt.normal_plot(df_diff, xlim_range, vert_line_date, quantile, 'k-o')     
    #myplt.plot_hist_returns(basket_x_days_hist_rtns, df_buy_sell_signal, xlim_range, LOWER_THRESHOLD, vert_line_date) 

### PnL Performance

In [ ]:
start_date = '2020-05-14'
end_date = TODAY

traded_fx_rates = df_fx_name_list_XXXUSD.loc[start_date]

num_rows = df_MeanRevStats['Return Expectation over X days'].loc[start_date:end_date].count()

### if using constant notional 
constant_notional = -2000000

# get the size of the list correct
constant_notionals = np.ones(num_rows) * constant_notional

output_xls = '../PnL Analysis from ' + start_date + ' to ' + end_date + '.xlsx'

df_pnl = ut.pnl_analysis_info_to_Excel(df_fx_name_list_XXXUSD, 
                                       constant_notionals, 
                                       weights_XXXUSD,
                                       traded_fx_rates, 
                                       start_date, 
                                       end_date)



df_pnl.to_excel(output_xls)

df_pnl['Basket Cumulative PnL'].plot(grid=True, figsize=(15,5))